In [4]:
import sys
from sqlitedict import SqliteDict
db = SqliteDict("Hansard.sqlite")
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
STOPWORDS.add('(b)')
STOPWORDS.add('(a)')
STOPWORDS.add('(c)')
extra_stopwords = ['','b', 'will','ask','make','use', 'state','secretary','hon','members','minister','take','say','many','point','statement','go','(a)','may','great','give','one','years','people','right','member','government','house','need','much','friend','come','()','now','mean','reduce','agree','us','(c)','mr','word','want','tell','end','whether','two','£','%','see','gentleman','put','long','yet']
stopwords = set(STOPWORDS)
new_stopwords = stopwords.copy()
for w in extra_stopwords:
    new_stopwords.add(w)
stopwords = new_stopwords

In [18]:
list(db.keys())

['25165', '10505', '24910']

In [25]:
data = db[24910]
speeches = data['speeches']

In [26]:
import string
from nltk.stem import WordNetLemmatizer
import re
def preprocess_speeches(speeches, stopwords = [], min_length = 25):
    #Removes all punctuation
    text = [t.translate(str.maketrans('', '', string.punctuation)) for t in speeches]
    #Removes text inside parenthesis
    text = [re.sub(r'\d+', '', t) for t in text]
    #Converts text into lowercase, and splits by whitespace
    text = [t.lower() for t in text]
    text = [' '.join([w for w in t.split() if not w in stopwords]) for t in text]
    text = [t for t in text if len(t.split()) >= min_length]
    return text

In [27]:
from bertopic import BERTopic
from umap import UMAP

In [28]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True, nr_topics=11)

In [29]:
# Run BERTopic model
text = preprocess_speeches(speeches, stopwords = stopwords, min_length = 25)
topics, probabilities = topic_model.fit_transform(text)

In [30]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1007,-1_energy_bill_uk_climate
1,0,993,0_eu_uk_department_health
2,1,626,1_energy_climate_nuclear_change
3,2,329,2_tax_exchequer_chancellor_work
4,3,154,3_vote_voting_mps_system
5,4,126,4_housing_local_rent_homes
6,5,125,5_education_women_schools_violence
7,6,108,6_rail_transport_trains_preston
8,7,59,7_green_deal_spending_equality
9,8,55,8_badgers_tb_badger_cattle
